# 章 データの集約とグループ演算

p315  
__データをカテゴライズして、それぞれのグループに関数を適用することは、集約や変換と呼ばれ、データ分析のワークフローの中で最も重要な部分。__  
この章では、

- pandasのオブジェクトを1つあるいは複数のキー（関数や配列、データフレームの列名の形式で指定を使って分割する方法。
- グループの要約統計量の計算方法。具体的には、個数のカウント、平均値、標準偏差や、ユーザが定義した関数などの計算方法。
- グループ内の変換やその他のデータ操作（正規化、線形回帰、順位、部分集合の選択）。
- ピボットテーブルとクロス集計の計算。
- 分位点分析やその他のデータから生成されたグループに関する分析。

を学ぶ。

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupByの仕組み

p316
- グループ演算のプロセスは__分離－適用－結合（split-apply-combine）__
- __分離__ は、pandasのオブジェクト（シリーズやデータフレームなど）を1つ以上のキーによって分割する事。
- __適用__ は、分割された各グループに関数が適用し、新しい値を生成する事。
- __結合__ は、適用結果が結合されて返却される事。


<img src="./image/chapter_10_001.jpg" alt="グループ集約操作の図解" title="グループ集約操作の図解" width="500" height="300">

__key1のラベルでグループ化して、data1列の平均値を計算する。__

In [2]:
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


p317
- グループ化された変数は、pandas.core.groupby.groupby.SeriesGroupByオブジェクトになる。
- SeriesGroupByオブジェクトには計算結果は含まれず、グループキー（例では df['key1']）に関する中間データが含まれている。

In [3]:
# data1列をkey1毎にグループ化
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# 平均
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

__グループキーに、複数の配列をリストとして渡す場合。__    
渡す順番は重要。

In [5]:
# グループキーを df['key1'、df['key2'] とする。
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [6]:
# グループキーを df['key1'、df['key2'] とする。
means = df['data1'].groupby([df['key2'], df['key1']]).mean()
means

key2  key1
one   a       0.880536
      b      -0.519439
two   a       0.478943
      b      -0.555730
Name: data1, dtype: float64

In [7]:
# key1を行へ移動
means.unstack()

key1,a,b
key2,,
one,0.880536,-0.519439
two,0.478943,-0.555730


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

display(df['data1'], states, years)

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
Name: data1, dtype: float64

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

array([2005, 2005, 2006, 2005, 2006])

p318  
__states、yearsの要素と、df1 のインデックス番号を割り当てる。__

 -0.204708　Ohio　2005  
 0.478943　California　2005  
 -0.519439　California　2006  
 -0.555730　Ohio　2005  
 1.965781　Ohio　2006

In [9]:
# data1列 の値と インデックスが被る states、years 要素を紐づけて平均を出す。
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [10]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [11]:
# key1 でグルーピングして平均を算出
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [12]:
# (a、data1）の場合
(-0.204708 + 0.478943 + 1.965781) / 3

0.7466719999999999

In [13]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

p319
- __GroupByオブジェクト__の __`size`メソッド__ は、各グループのサイズ情報を持つシリーズを戻す。

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### グループをまたいだ繰り返し

p319
- __`GroupBy`オブジェクト__は繰り返しをサポートする。
- 繰り返しの中では、グループの名前（name）と、その名前に対応するデータ（group）、のタプルを取得できる。

In [15]:
for name, group in df.groupby('key1'):
    print("### {} \n {}\n".format(name, group))

### a 
   key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435

### b 
   key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023



p320
- 複数キーを扱うケースでは、繰り返しの中で使用されるタプルに含まれる最初の要素は、キーの値のタプル。

In [16]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print("### {} - {} \n {}\n".format(k1, k2, group))

### a - one 
   key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435

### a - two 
   key1 key2     data1     data2
1    a  two  0.478943  0.092908

### b - one 
   key1 key2     data1     data2
2    b  one -0.519439  0.281746

### b - two 
   key1 key2    data1     data2
3    b  two -0.55573  0.769023



p320
- グルーピング結果からデータの取得にはディクショナリが使える。

In [17]:
# 各リスト要素の先頭はグループの名前(a,b）
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one -0.204708  1.393406
  1    a  two  0.478943  0.092908
  4    a  one  1.965781  1.246435), ('b',   key1 key2     data1     data2
  2    b  one -0.519439  0.281746
  3    b  two -0.555730  0.769023)]

In [18]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


p320
- デフォルトの__`groupby`__のグループは__`axis=0`__に適用される。
- __`dtype`__を使ってグループ化する事が出来る。

In [19]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [20]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
# dtypeを使って、axis=1 方向にグルーピング
grouped = df.groupby(df.dtypes, axis=1)
grouped

In [22]:
for dtype, group in grouped:
    print("### {}\n {}\n".format(dtype, group))

### float64
       data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435

### object
   key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one



### 列や列の集合の選択

p321 
- __DataFrameGroupBy__ に対してリストや配列でインデックス参照した場合はグループ化されたデータフレーム。
- スカラーとして単独の列名を与えた場合には、グループ化されたシリーズ。

In [23]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [24]:
# 単独の列名を与えた場合はシリーズ
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

In [25]:
# リストや配列を与えた場合はデータフレーム
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

### ディクショナリやシリーズのグループ化

p322
- __groupby__ にディクショナリを渡してグルーピングが可能。
- 固定サイズマッピングで__groupby__ にシリーズ渡してもグルーピングが可能。  
（実際は、関数を使ってグルーピングする方が汎用的）

In [26]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=['a', 'b', 'c', 'd', 'e'],
    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan  # Add a few NA values
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [27]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f': 'orange'
}

mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [28]:
# ディクショナリを使用してグルーピング（orangeはグルーピングされていない事に注意）
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [29]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
# シリーズを利用したグルーピング（orangeはグルーピングされていない事に注意）
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 関数を使ったグループ化

p324
- ディクショナリやシリーズを使ったグルーピングより、関数を使う方法は、より汎用的。
- グループキーとして渡される関数は、インデックスの値ごとに呼び出される。
- 関数の戻り値がグループ名として用いられる。

In [31]:
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [32]:
# 行インデックスの値に対してlen関数を適用 （例：len("joe")　）
people.groupby(len).sum()

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


下記は、どう理解すれば良いのか？？？

In [33]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

### インデックス階層によるグループ化

p324
- 階層インデックスによる軸の集約は強力。
- 階層ごとに集約するには、階層の番号やlevelキーワードを使って階層を指定する。

In [34]:
columns = pd.MultiIndex.from_arrays(
    [['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.560145 -1.265934  0.119827 -1.063512  0.332883
1     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
2      0.286350  0.377984 -0.753887  0.331286  1.349742
3      0.069877  0.246674 -0.011862  1.004812  1.327195

In [35]:
# city 列（= US,JP)でカウント
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## データの集約

p325
- 集約とは、何らかのデータ変形を行って配列からスカラー値を生成する事。
- グループ化されたオブジェクトに対して__`quantile`メソッド__で分位点を計算する事も可能。

p326
- 自分自身で定義した集約関数を使うには、配列を集約する関数を__`aggregate`__あるいは__`agg`メソッド__に渡す。
- 独自に作成した集約関数は、下記の__最適化されたGroupByメソッド__と比べて、一般的に遅い。

__最適化されたGroupByメソッド__

- __`count`__ 　　　グループ内の欠損値以外の値の数。
- __`sum`__ 　　　　欠損値以外の合計。
- __`mean`__ 　　　　欠損値以外の平均。
- __`median`__ 　　　欠損値以外の算術中央値。
- __`std, var`__ 　　バイアスの掛かっていない（n－1を分母とした）標準偏差と分散。
- __`min, max`__ 　　欠損値以外の最小値と最大値。
- __`prod`__ 　　　　欠損値以外の積。
- __`first, last`__ 　欠損値以外の最初と最後の値。

In [36]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [37]:
# グループ化されたされたオブジェクトに対して、90%分位点を算出
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [38]:
# aの 50% タイルは 0.478943
(1.965781 - 0.478943) * 4 / 5 + 0.478943

1.6684134

In [39]:
# b の 90% タイル
-0.555730 + (-0.519439 + 0.555730) * 0.9

-0.5230680999999999

__自分で定義した関数を使用する場合__  
（一般的に最適化されたGroupByメソッドの方が早い。）

In [40]:
# 最大値 - 最小値
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [41]:
# a - data1 の　最大値 - 最小値
1.965781 - (-0.204708)

2.170489

In [42]:
# グループに対して要約統計
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

### 列に複数の関数を適用する

p327
- 列ごとに異なる関数や関数の名前のリストを指定した場合、戻り値として関数名と同じ列名を持つデータフレームを得る。

p328
- __lambda関数__ を適用したときに列に付けられる名前は __`<lambda>`__
- 関数の_ __name__ _属性を見ると、どのような名前が列名に使われるのか確認することができる。
- __( 名前, 関数)__という形式のタプルをリストで渡せば、各タプルの最初の要素が列名として使われる。


In [43]:
tips = pd.read_csv('examples/tips.csv')

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [46]:
# dayとsmokerでグルーピング
grouped = tips.groupby(['day', 'smoker'])

# グルーピングされた tip_pct列 を参照
grouped_pct = grouped['tip_pct']

# 平均を計算
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [48]:
# mean と std列名 を持つデータフレームを返却（peak_to_peak は自作関数）
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [50]:
# タプルの最初の要素が列名として使用される
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [52]:
functions = ['count', 'mean', 'max']

# グルーピングされた tip_pct、total_bill 列に対して count、mean、max を適用
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [53]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

__例）関数にデータフレームの列名を指定する__

In [59]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [57]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

__例）複合インデックスを作成する__

In [60]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 集約されたデータを行インデックスなしで戻す

p330
- __`as_index_=False`__ をgroupbyに渡すことで、ほとんどの場合に集約されたデータをインデックスなしに出来る。

In [62]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [71]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [69]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [72]:
tips.groupby(['day', 'smoker']).mean().columns, tips.groupby(['day', 'smoker'], as_index=False).mean().columns

(Index(['total_bill', 'tip', 'size', 'tip_pct'], dtype='object'),
 Index(['day', 'smoker', 'total_bill', 'tip', 'size', 'tip_pct'], dtype='object'))

__`as_index=False`__を用いると、__groupby__ に渡した day、smokerが 行ラベル ではなく 列ラベル になる。

## Apply: General split-apply-combine

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result
result.unstack('smoker')

f = lambda x: x.describe()
grouped.apply(f)

### Suppressing the Group Keys

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

### Quantile and Bucket Analysis

In [ ]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

In [ ]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

### Example: Filling Missing Values with Group-Specific       Values

In [ ]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
data.groupby(group_key).mean()

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

### Example: Random Sampling and Permutation

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [ ]:
deck[:13]

In [ ]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

In [ ]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

### Example: Group Weighted Average and Correlation

In [ ]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

In [ ]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [ ]:
rets = close_px.pct_change().dropna()

In [ ]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Example: Group-Wise Linear Regression

In [ ]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

## Pivot Tables and Cross-Tabulation

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

### Cross-Tabulations: Crosstab

In [ ]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [ ]:
data

In [ ]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion